<a href="https://colab.research.google.com/github/paolopdel/Lesson_8_17_char_RNN_songs/blob/master/Lesson8_17_char_Lev_Rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [0]:
# open text file and read in data as `text`
with open('data/Output1.txt', 'r') as f:
    text = f.read()

In [90]:
text[:100]
#len(text)


"\ufeffDoesn't take much to make me happy\nAnd make me smile with glee \nNever never will I feel discouraged"

In [91]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
print(chars)
int2char = dict(enumerate(chars))
print(int2char)
char2int = {ch: ii for ii, ch in int2char.items()}
print(char2int)
# encode the text
encoded = np.array([char2int[ch] for ch in text])
len(chars)


('m', '”', 'é', "'", 'w', 'c', '-', '&', '値', 'S', 'D', 'U', 'f', 'j', '1', '[', 'H', 'n', '"', '“', 'C', 'W', 'B', 'O', 'u', 'ì', 's', '4', 'x', '—', 'y', 'V', '’', '7', 'r', '}', '`', '\n', 'k', ')', 'M', 'q', '貓', 'K', 'i', ' ', '.', 'h', '_', 'R', 'è', 'F', '…', ';', 'd', 'P', 'Q', 'o', '#', 'Y', ',', 't', 'È', '‘', '2', '\\', '\ufeff', '8', 'à', '3', '0', 'z', 'b', 'p', 'v', 'a', '!', 'I', '(', '{', 'l', ':', 'ù', '5', '´', 'T', 'E', '9', '/', 'ç', ']', '\t', '?', '6', 'e', '~', 'A', 'Z', 'ã', 'G', 'J', 'g', 'N', '王', 'L')
{0: 'm', 1: '”', 2: 'é', 3: "'", 4: 'w', 5: 'c', 6: '-', 7: '&', 8: '値', 9: 'S', 10: 'D', 11: 'U', 12: 'f', 13: 'j', 14: '1', 15: '[', 16: 'H', 17: 'n', 18: '"', 19: '“', 20: 'C', 21: 'W', 22: 'B', 23: 'O', 24: 'u', 25: 'ì', 26: 's', 27: '4', 28: 'x', 29: '—', 30: 'y', 31: 'V', 32: '’', 33: '7', 34: 'r', 35: '}', 36: '`', 37: '\n', 38: 'k', 39: ')', 40: 'M', 41: 'q', 42: '貓', 43: 'K', 44: 'i', 45: ' ', 46: '.', 47: 'h', 48: '_', 49: 'R', 50: 'è', 51: 'F', 52: '…

105

In [92]:
encoded[:100]

array([ 66,  10,  57,  94,  26,  17,   3,  61,  45,  61,  75,  38,  94,
        45,   0,  24,   5,  47,  45,  61,  57,  45,   0,  75,  38,  94,
        45,   0,  94,  45,  47,  75,  73,  73,  30,  37,  96,  17,  54,
        45,   0,  75,  38,  94,  45,   0,  94,  45,  26,   0,  44,  80,
        94,  45,   4,  44,  61,  47,  45, 101,  80,  94,  94,  45,  37,
       102,  94,  74,  94,  34,  45,  17,  94,  74,  94,  34,  45,   4,
        44,  80,  80,  45,  77,  45,  12,  94,  94,  80,  45,  54,  44,
        26,   5,  57,  24,  34,  75, 101,  94,  54])

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    

#crea un array: 3 con aggiuna 8 (3,8)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.

    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot


In [94]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)
#print(test_seq)   #[[3 5 1]]
#print(test_seq.shape)#(1, 3)
#print(test_seq.size)   #3
#print(test_seq.flatten())  #[3 5 1]
#print(test_seq) #[[3 5 1]]
#arange che si trova nel codice sopra è come for, ma parte da dove si vuole, primo argomento, passo nel secondo arg.
#l'array 3,5,1 viene posto come primo arg. 

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [0]:
#starting sequence: [1 2 3 4 5 6 7 8 9 10 11 12]
#Batch size: 2   [1 2 3 4 5 6 ]
#               [7 8 9 10 11 12]
#sequence lenght: 3  [1 2 3 ......
#                    [7 8 9 ......


In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [96]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)
print(batches)
1985223//400

<generator object get_batches at 0x7fa2e72cc1a8>


4963

In [97]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 66  10  57  94  26  17   3  61  45  61]
 [ 24  45  75  34  94  45   0  30  45  80]
 [ 61  47  94  45  72  34  94  94  71  94]
 [ 74  94  34  45  80  94  61  45  30  57]
 [ 45  61  94  80  80  45  30  57  24  45]
 [ 57  24  61  45  30  57  24  37  37  99]
 [101  44  34  80  45  80  57  57  38  45]
 [101  37  37   9  75  30  45   0  30  45]]

y
 [[10 57 94 26 17  3 61 45 61 75]
 [45 75 34 94 45  0 30 45 80 75]
 [47 94 45 72 34 94 94 71 94 26]
 [94 34 45 80 94 61 45 30 57 24]
 [61 94 80 80 45 30 57 24 45 75]
 [24 61 45 30 57 24 37 37 99 44]
 [44 34 80 45 80 57 57 38 45 75]
 [37 37  9 75 30 45  0 30 45 17]]


In [98]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            #h = tuple([each.data for each in h])
            h = tuple([each.data for each in h])
            #print(h)
            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                   
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))



In [101]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net.chars)
len(net.chars)


('m', '”', 'é', "'", 'w', 'c', '-', '&', '値', 'S', 'D', 'U', 'f', 'j', '1', '[', 'H', 'n', '"', '“', 'C', 'W', 'B', 'O', 'u', 'ì', 's', '4', 'x', '—', 'y', 'V', '’', '7', 'r', '}', '`', '\n', 'k', ')', 'M', 'q', '貓', 'K', 'i', ' ', '.', 'h', '_', 'R', 'è', 'F', '…', ';', 'd', 'P', 'Q', 'o', '#', 'Y', ',', 't', 'È', '‘', '2', '\\', '\ufeff', '8', 'à', '3', '0', 'z', 'b', 'p', 'v', 'a', '!', 'I', '(', '{', 'l', ':', 'ù', '5', '´', 'T', 'E', '9', '/', 'ç', ']', '\t', '?', '6', 'e', '~', 'A', 'Z', 'ã', 'G', 'J', 'g', 'N', '王', 'L')


105

In [102]:
#batch_size = 128
#seq_length = 100
#n_epochs = 20 # start smaller if you are just testing initial behavior

batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior lr=0.001

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.3030... Val Loss: 3.2974
Epoch: 1/20... Step: 20... Loss: 3.2211... Val Loss: 3.2124
Epoch: 1/20... Step: 30... Loss: 3.1805... Val Loss: 3.2064
Epoch: 1/20... Step: 40... Loss: 3.1665... Val Loss: 3.2025
Epoch: 1/20... Step: 50... Loss: 3.1370... Val Loss: 3.2034
Epoch: 1/20... Step: 60... Loss: 3.1551... Val Loss: 3.2020
Epoch: 2/20... Step: 70... Loss: 3.1512... Val Loss: 3.1985
Epoch: 2/20... Step: 80... Loss: 3.1428... Val Loss: 3.1958
Epoch: 2/20... Step: 90... Loss: 3.1300... Val Loss: 3.1918
Epoch: 2/20... Step: 100... Loss: 3.1197... Val Loss: 3.1814
Epoch: 2/20... Step: 110... Loss: 3.0938... Val Loss: 3.1499
Epoch: 2/20... Step: 120... Loss: 3.0261... Val Loss: 3.0814
Epoch: 3/20... Step: 130... Loss: 2.9963... Val Loss: 3.0358
Epoch: 3/20... Step: 140... Loss: 2.8764... Val Loss: 2.9094
Epoch: 3/20... Step: 150... Loss: 2.7594... Val Loss: 2.8154
Epoch: 3/20... Step: 160... Loss: 2.6530... Val Loss: 2.7242
Epoch: 3/20... Step: 170... Loss:

In [0]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        #print(x)
        x = one_hot_encode(x, len(net.chars))
        #print(x)
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            #print(top_ch)
            #print(p.topk(top_k))
            top_ch = top_ch.numpy().squeeze()
            #print(top_ch)
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        #print(char)
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='Milano sembra Rio', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [138]:
print(sample(net, 5000, prime='Milano sembra Rio', top_k=5))

Milano sembra Riother sterness
We couldn't make it to me 
You are the thought oh, time, you're my lover

You know you'll live, you and me all you're makes
You're to me the morning our heart,, I never been thinking for a little dire
She dail
I don't'll be your
I'll be lowe to yee

If you can't give me who waiting for all my life
I don't wanna believe in things
And the way you see it what you're seeming means 
I'm the one that I want my heart a love find
I've got to love you with me in my shile
It was love

The world and going to terstit me and then I love you all
I'm tilling you are the one that I could never be my life

I've been there with you 
And I've been strend a wonlore 
To the sand to too so but of all I can't do

And that I walk a morning
And I'm stoply for love 
It's and love if you're always like you 

You're said, I can't wait for lovin' you, baby
And I want you're that I love you
And I, I won't love you so beat

All the way you see the darking of your eyes
You are the way t